In [1]:
# run once (may have to restart notebook)
# !pip install tensorflow-gpu==1.15 --user
# !pip install gpt-2-simple --user

In [2]:
import pandas as pd
import numpy as np
import json
import gpt_2_simple as gpt2
import os
import requests

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [14]:
# check to make sure gpu is recognized for significantly faster training

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [7]:
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)

In [8]:
dir_path = "../data/"

dem_file_name = "democrats_result.txt"
dem_sample_name = "democrats_sample.txt"

rep_file_name = 'republican_result.txt'
rep_sample_name = 'republican_sample.txt'

both_file_name = 'both_result.txt'
both_sample_name = 'both_sample.txt'

In [9]:
# reading and writing sample files for each side

# with open(dir_path+dem_file_name,'r') as demf:
#     dem_data = demf.readlines(100000)
# with open(dir_path+dem_sample_name,'w+') as dem_write:
#     dem_write.writelines(dem_data)

# with open(dir_path+rep_file_name,'r') as repf:
#     rep_data = repf.readlines(100000)
# with open(dir_path+rep_sample_name,'w+') as rep_write:
#     rep_write.writelines(rep_data)

# with open(dir_path+both_file_name,'r') as bothf:
#     both_data = bothf.readlines(100000)
# with open(dir_path+both_sample_name,'r') as both_write:
#     both_write.writelines(both_data)

In [10]:
# text file to train model on

train_fp = dir_path + dem_sample_name

In [11]:
# this cell takes the longest. Can only be run once without restarting the notebook
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              train_fp,
              model_name=model_name,
              steps=100, # steps is max number of training steps
              restore_from='fresh', # makes sure model doesnt resume from previous trained model
              print_every=20 # only prints every 20 training steps
             )   

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint/run1/model-100
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-100


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  2.76it/s]


dataset has 23191 tokens
Training...
======== SAMPLE 1 ========
 the White House. But if we control both houses of Congress and the White House we can take ownership. And Democrats can't even be elected. We can't even be President. *https://en.m.wikipedia.org/wiki/International_Protestant</a>* *https://en.pa.gov/ranges/resistance/index.sh?hl=fr&b=17001&12391a&oldview> **Protest Like No Other People** Public protests are the most effective way to anger and control the government. No democratic government can be expected to provide for the family when its wintering in an out-of-home father of three in his home state of Wisconsin. 


This type of social mobility is impossible in most of asia, and much of the rest of the world as well. So other average people in other countries are dying of hunger, getting into literal slavery, doing backbreaking work in mines - they do NOT have it better than us. My family's neighbors have NO food right now. My dad's best friend who he grew up with is lit

In [12]:
# prompt to generate response to 

pre = "After witnessing the heinous amount of police brutality in the US against POC, \
my partner and I have created some protest posters in response. We do not care about \
credit, just getting the message out there, so feel free to distribute however way you \
want, or make better versions yourself if you think it can be improved!"

In [13]:
gpt2.generate(sess, 
              temperature=.7, # uniqueness of the output (usually ranges from .5 to 2)
              prefix=pre, # prompt
              nsamples=5, # number of generated responses 
              length=400 # number of words (including prompt) per response
             )

After witnessing the heinous amount of police brutality in the US against POC, my partner and I have created some protest posters in response. We do not care about credit, just getting the message out there, so feel free to distribute however way you want, or make better versions yourself if you think it can be improved! I understand the anger and the angering responses, but only one can make a difference! I'll see you then...


Peace and blessings.

I'll see you then! I'll see you soon!

I pray that by sharing this with as many people as possible I think will help you Grow Strong Again! Thank you!!! Keep up the great work! Thanks for nothing, just awesome. Everyone is awesome!! I'm sure as hell not even close! Who doesn't love a good story.  Great job sharing this on Reddit! I've started looking for work in the real estate market. Thanks for nothing, just awesome. Everyone is awesome!! One of the most widely admired people you will ever meet. One of the most widely admired people you 

After witnessing the heinous amount of police brutality in the US against POC, my partner and I have created some protest posters in response. We do not care about credit, just getting the message out there, so feel free to distribute however way you want, or make better versions yourself if you think it can be improved! I understand sentimentally, I don't. I'm not  a criminal justice system expert, but if you could give us an example of an example you feel is most clearly being used by the protestors, would you give it a tryst if called upon to do so? Anything that would make you think twice before doing so, especially since you clearly do not understand the point of protesting. Instead, you resort to violence, selfish rage bing, buzzwords like "terrorist", and so on. That's selfish, stupid, and often violent. That's violent, vicious, and unhinged. That's unhinged. That's destructive. That's sad. But also normal. I don’t know of a single one of those millions. It's a small minority th